## XOR with logistic refression

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.loadtxt.html
xy = np.loadtxt('train_data/lab9-1_train.txt', unpack=True)
x_data = np.transpose(xy[0:-1])
y_data = np.reshape(xy[-1], (4,1))

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([1, len(x_data)], -1.0, 1.0))

# Our hypothesis
h = tf.matmul(W, X)
hypothesis = tf.div(1., 1.+tf.exp(-h))
# cost function
cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y) * tf.log(1.-hypothesis))

# Minimize
a = tf.Variable(0.01) # Learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# Before starting, initialize the bariables. We will 'run' this first.
init = tf.global_variables_initializer()

In [3]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # fit the line.
    for step in range(1000):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}), sess.run(W))
            
    # Test model
    correct_prediction = tf.equal(tf.floor(hypothesis + 0.5), Y)
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(sess.run([hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], feed_dict={X:x_data, Y:y_data}))
    print("Accuracy:", accuracy.eval({X:x_data, Y:y_data}))

0 0.695131 [[-0.54783797 -0.7434423  -0.99530113  0.87447751]]
200 0.69435 [[-0.54783797 -0.77218461 -0.96835333  0.87268329]]
400 0.693877 [[-0.54783797 -0.79430526 -0.94708174  0.87183428]]
600 0.693589 [[-0.54783797 -0.81140679 -0.93038201  0.87143278]]
800 0.693415 [[-0.54783797 -0.82466477 -0.91731369  0.87124342]]
[array([[ 0.49100143,  0.50905883]], dtype=float32), array([[ 0.,  1.]], dtype=float32), array([[ True, False],
       [False,  True],
       [False,  True],
       [ True, False]], dtype=bool), 0.5]
Accuracy: 0.5


### Does not work!
우리가 예상한 값은 [0,1,1,0]이지만 결과는 [1,1,0,0]이 나옴  
50% 밖에 맞지 않음 

## NN for XOR

In [4]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [5]:
W1 = tf.Variable(tf.random_uniform([2,2], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform([2,1], -1.0, 1.0))

b1 = tf.Variable(tf.zeros([2]), name="Bias1")
b2 = tf.Variable(tf.zeros([1]), name="Bias2")

# Our hypothesis
L2 = tf.sigmoid(tf.matmul(X, W1) + b1)
hypothesis = tf.sigmoid(tf.matmul(L2, W2) + b2)

In [6]:
# cost function
cost = -tf.reduce_mean( Y*tf.log(hypothesis)+(1-Y)* tf.log(1.-hypothesis) )

# Minimize
a = tf.Variable(0.1) # Learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# Before starting, initialize the bariables. We will 'run' this first.
init = tf.global_variables_initializer()

In [7]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # fit the line.
    for step in range(20001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 4000 == 0:
            print (step, sess.run(cost, feed_dict={X:x_data, Y:y_data}), sess.run(W1), sess.run(W2))
            
    # Test model
    correct_prediction = tf.equal(tf.floor(hypothesis+0.5), Y)
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print( sess.run( [hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy],
                   feed_dict={X:x_data, Y:y_data}) )
    print("Accuracy:", accuracy.eval({X:x_data, Y:y_data}))

0 0.720215 [[ 0.87414038 -0.51383328]
 [ 0.6796999   0.63277024]] [[ 0.60214341]
 [ 0.18827695]]
4000 0.387929 [[ 4.90775537  0.61399132]
 [ 5.60549736  2.41313958]] [[ 5.5677042]
 [-3.6628325]]
8000 0.0278565 [[ 6.38019085  4.09327745]
 [ 6.64880466  4.12744379]] [[ 8.84407139]
 [-9.50091362]]
12000 0.0131026 [[ 6.77163839  4.6391139 ]
 [ 6.958076    4.66630936]] [[ 10.12051582]
 [-10.91417217]]
16000 0.00849369 [[ 6.99005032  4.92252922]
 [ 7.14098024  4.94578981]] [[ 10.88570213]
 [-11.71309376]]
20000 0.00626431 [[ 7.13969326  5.11047363]
 [ 7.2699542   5.13115549]] [[ 11.43267536]
 [-12.27146339]]
[array([[ 0.00766805],
       [ 0.99439323],
       [ 0.99432111],
       [ 0.00602377]], dtype=float32), array([[ 0.],
       [ 1.],
       [ 1.],
       [ 0.]], dtype=float32), array([[ True],
       [ True],
       [ True],
       [ True]], dtype=bool), 1.0]
Accuracy: 1.0


## Wide NN for XOR

In [8]:
# Wide network: Use more neurons in each layer. 
W1 = tf.Variable(tf.random_uniform( [2,10], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform( [10,1], -1.0, 1.0))

b1 = tf.Variable(tf.zeros([10]), name="Bias1")
b2 = tf.Variable(tf.zeros([1]), name="Bias2")

In [9]:
# Hypotheses 
L2 =  tf.sigmoid(tf.matmul(X,W1)+b1)
hypothesis = tf.sigmoid( tf.matmul(L2,W2) + b2)

# Cost function 
cost = -tf.reduce_mean( Y*tf.log(hypothesis)+(1-Y)* tf.log(1.-hypothesis) )

# Minimize cost.
a = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# Initializa all variables.
init = tf.global_variables_initializer()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(20001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        
        if step % 4000 == 0:
            print(
                step, 
                sess.run(cost, feed_dict={X:x_data, Y:y_data}), 
                sess.run(W1),
                sess.run(W2)
            )
    
    # Test model
    correct_prediction = tf.equal( tf.floor(hypothesis+0.5), Y)
    accuracy = tf.reduce_mean(tf.cast( correct_prediction, "float" ) )
    
    # Check accuracy
    print( sess.run( [hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], 
                   feed_dict={X:x_data, Y:y_data}) )
    print( "Accuracy:", accuracy.eval({X:x_data, Y:y_data}) )

0 0.696199 [[-0.60511547 -0.30814201  0.72101784 -0.44072786 -0.52743822 -0.52494985
   0.82023078 -0.47523519  0.45831645  0.44436476]
 [ 0.56247801  0.44132757  0.75539881 -0.34457281 -0.01685247 -0.72980714
  -0.93928814  0.99081457  0.62967372 -0.59601557]] [[ 0.19279987]
 [ 0.30260223]
 [ 0.50413382]
 [-0.8679266 ]
 [-0.86838031]
 [-0.94578373]
 [ 0.0179239 ]
 [ 0.29100806]
 [ 0.16387145]
 [ 0.39473936]]
4000 0.0748434 [[-3.98894525 -2.04667044  3.19953847  0.86206311  1.11588979 -4.96113443
   0.95689529 -0.25528294  1.10183704  1.88897526]
 [ 2.22388458  0.3829039   3.26879835  0.5600692   1.32530522 -5.0237813
  -2.44750023  1.60399044  1.04467309 -3.64847136]] [[ 4.21929455]
 [ 1.94129074]
 [ 3.00831676]
 [-1.49375117]
 [-2.83520269]
 [-6.59566641]
 [ 1.92231452]
 [-1.26308572]
 [-2.24695611]
 [ 3.63915038]]
8000 0.0126279 [[-4.93072414 -2.63586497  3.67633724  1.06216931  1.71939552 -5.56724644
   1.49972129 -0.41105032  1.4885267   2.85678005]
 [ 3.27815628  0.78718275  3.73

## Deep NN for XOR

In [10]:
# Deep network configuration.: Use more layers. 
W1 = tf.Variable(tf.random_uniform( [2,5], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform( [5,4], -1.0, 1.0))
W3 = tf.Variable(tf.random_uniform( [4,1], -1.0, 1.0))


b1 = tf.Variable(tf.zeros([5]), name="Bias1")
b2 = tf.Variable(tf.zeros([4]), name="Bias2")
b3 = tf.Variable(tf.zeros([1]), name="Bias3")

In [11]:
# Hypotheses 
L2 =  tf.sigmoid(tf.matmul(X,W1)+b1)
L3 =  tf.sigmoid(tf.matmul(L2,W2)+b2)
hypothesis = tf.sigmoid( tf.matmul(L3,W3) + b3)

In [12]:
# Cost function 
cost = -tf.reduce_mean( Y*tf.log(hypothesis)+(1-Y)* tf.log(1.-hypothesis) )

# Minimize cost.
a = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# Initializa all variables.
init = tf.global_variables_initializer()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(20001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        
        if step % 4000 == 0:
            print(
                step, 
                sess.run(cost, feed_dict={X:x_data, Y:y_data}), 
                sess.run(W1),
                sess.run(W2)
            )
    
    # Test model
    correct_prediction = tf.equal( tf.floor(hypothesis+0.5), Y)
    accuracy = tf.reduce_mean(tf.cast( correct_prediction, "float" ) )
    
    # Check accuracy
    print( sess.run( [hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], 
                   feed_dict={X:x_data, Y:y_data}) )
    print( "Accuracy:", accuracy.eval({X:x_data, Y:y_data}) )

0 0.771545 [[-0.52903324  0.69870329 -0.03112978 -0.84536332  0.0306104 ]
 [ 0.49851146 -0.55360097 -0.25324705 -0.96951008 -0.05127549]] [[-0.49905097  0.21191521  0.94059336 -0.18701103]
 [ 0.02787192 -0.63945204  0.89419991 -0.99158889]
 [-0.51910573 -0.83378035 -0.31925189 -0.69106519]
 [ 0.84683406 -0.43074334  0.28659174 -0.61934024]
 [-0.22357935  0.66982335  0.64668    -0.21907875]]
4000 0.565237 [[-0.46245572  2.25516248  0.12671506 -2.91392279  0.10296836]
 [ 0.54753196 -1.09154069 -0.42812011 -2.78314757 -0.05475603]] [[-0.37615064  0.16874325  0.73164707  0.26765952]
 [ 0.23662229 -0.53476053  0.90925115 -1.46448863]
 [-0.61298555 -0.88003689 -0.68644118 -0.14716712]
 [ 1.8068856   0.01620655  1.29206204 -2.12067866]
 [-0.25306392  0.63263464  0.34413883  0.20570481]]
8000 0.00895151 [[-0.24447788  5.1082058   1.11432636 -5.06388283 -0.15630572]
 [ 1.15111411 -3.27306223 -3.96538782 -4.79304743 -1.33744121]] [[-0.15568954  0.19045594  0.74721485 -0.35238701]
 [ 2.2063961  -